In [1]:
import ee
ee.Initialize()
import geemap
Map = geemap.Map()

In [2]:
from datetime import datetime

In [3]:
def addndvi(image):
    return image.addBands(image.normalizedDifference(['sur_refl_b02','sur_refl_b01']).rename('NDVI')).float()

In [4]:
primary = ee.ImageCollection('MODIS/006/MOD09A1').filterDate('2020-01-01', '2020-12-31').map(addndvi).select('NDVI')
era5d = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").filter(ee.Filter.date('2020-01-01', '2020-12-31')).select('temperature_2m')

In [80]:
oneday = 1*24*60*60*1000
timefilter = ee.Filter(
    ee.Filter.maxDifference(
        difference = oneday,
        leftField = 'system:time_start',
        rightField = 'system:time_end'))

saveallj = ee.Join.saveAll(
    matchesKey = 'era5',
    ordering = 'system:time_start',
    ascending = True
)
collj = ee.ImageCollection(saveallj.apply(primary,era5d,timefilter))

In [94]:
def temp_filt(image):
    temp = ee.ImageCollection(ee.List(image.get('era5'))).mean()
    temp1 = temp.expression('temperature_2m -273.15',{'temperature_2m': temp.select('temperature_2m')})
    return image.updateMask(temp1.gt(0)).select('NDVI')

In [95]:
finalcol = collj.map(temp_filt)

In [42]:
#for one in infol:
#    st = one.get('properties').get('system:time_start')
#    dt = datetime.fromtimestamp(st/1000).strftime('%c')
    #print(dt)

In [248]:
#datetime.fromtimestamp(1578528000000/1000).strftime('%c')

'Thu Jan  9 01:00:00 2020'

In [149]:
#day2 = day1.expression('temperature_2m -273.15',{'temperature_2m': day1.select('temperature_2m')})
#day2b = day2.updateMask(day2.gt(0))

In [100]:
#Map.addLayer(hh,{'palette':['blue','green','red']},'temp6')

In [29]:
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [137]:
ee.Date(image.date()).format('yyyy-MM-dd').getInfo()

'2020-01-01'

In [147]:
primary = ee.ImageCollection('MODIS/006/MOD09A1').filterDate('2020-01-01', '2020-01-02')
primary.size().getInfo()

1